In [47]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

In [48]:

def fetch_data():
    file_path = "company_esg_score.xlsx"  
    df = pd.read_excel(file_path, header=0)
    return df

In [49]:

def determine_esg_level(total_score):
    if total_score >= 85.72:
        return "AAA"
    elif total_score >= 71.44:
        return "AA"
    elif total_score >= 57.15:
        return "A"
    elif total_score >= 42.87:
        return "BBB"
    elif total_score >= 28.68:
        return "BB"
    elif total_score >= 14.30:
        return "B"
    else:
        return "CCC"

In [50]:
df = fetch_data()
e_score = df["E Score"].iloc[-1]
s_score = df["S Score"].iloc[-1]
g_score = df["G Score"].iloc[-1]

In [51]:
# indicators of three polar charts
e_metrics = ["Emission intensities", "Energy consumption intensity", "Waste generated", "Water intensity"]
s_metrics = ["Average training hours per employee", "Current employees by age groups (aged above 50 years old)",
             "Current employees by gender (female)", "Employee satisfaction rate",
             "New hires by gender (female)", "New hires by age (aged above 50 years old)",
             "Total turnover", "Turnover by gender (female)", "Turnover by age (aged above 50 years old)",
             "Fatalities", "Percentage of employees covered by health insurance",
             "High-consequence injuries", "Work-related injuries",  "Company donated"]
g_metrics = ["Board independence", "Women in the management team", "Women on the board"]

In [52]:
# horizontal bar chart of e_score, s_score and g_score
def create_bar_chart(e_score, s_score, g_score):
    fig_bar = go.Figure()
    fig_bar.add_trace(go.Bar(
        x = [e_score, s_score, g_score],
        y = ["Environment", "Social", "Governance"],
        marker = dict(color = "lightgreen"),
        orientation = "h"
))
    return fig_bar

In [53]:
# polar chart of indicators
def create_polar_chart(values, metrics, title):
    sorted_data = sorted(zip(values, metrics), reverse=True)
    sorted_values, sorted_metrics = zip(*sorted_data)
    fig = go.Figure(go.Barpolar(
        r = values,
        theta = metrics,
        marker = dict(
            color = values,
            colorscale = "Greens",  
            cmin = 0,
            cmax = max(values),
            line = dict(color = "darkgreen", width = 1)  
        ),
        text = metrics,
        hoverinfo = "text+r"
    ))

    fig.update_layout(
        title = title,
        polar = dict(
            radialaxis = dict(visible = True, range = [0, max(values)], showticklabels = False),  
            angularaxis = dict(showline = False, tickfont = dict(size=12), showticklabels = False)  
        ),
        showlegend = False
    )
    return fig

In [54]:
# Create a Dash app
app = Dash(__name__)

In [55]:
app.layout = html.Div([
    dcc.Graph(id = "polar-e-chart"),  
    dcc.Graph(id = "polar-s-chart"),  
    dcc.Graph(id = "polar-g-chart"),  
    dcc.Graph(id = "bar-chart"),  
    dcc.Input(id = "dummy-input", type = "hidden", value = "dummy")  
])

In [56]:
@app.callback(
    [Output("polar-e-chart", "figure"),
     Output("polar-s-chart", "figure"),
     Output("polar-g-chart", "figure"),
     Output("bar-chart", "figure")],
     Input("dummy-input", "value")
)

def update_charts(selected_metric):
    df = fetch_data()  
    e_score = df["E Score"].iloc[-1]
    s_score = df["S Score"].iloc[-1]
    g_score = df["G Score"].iloc[-1]
    
    bar_fig = create_bar_chart(e_score, s_score, g_score)
    
     # Polar chart of environment indicators
    e_values = [df[m].iloc[-1] for m in e_metrics]
    polar_e_fig = create_polar_chart(e_values, e_metrics, "Environment Metrics")

    # Polar chart of social indicators
    s_values = [df[m].iloc[-1] for m in s_metrics]
    polar_s_fig = create_polar_chart(s_values, s_metrics, "Social Metrics")

    # Polar chart of governance indicators
    g_values = [df[m].iloc[-1] for m in g_metrics]
    polar_g_fig = create_polar_chart(g_values, g_metrics, "Governance Metrics")
    
    return polar_e_fig, polar_s_fig, polar_g_fig, bar_fig

In [57]:
if __name__ == '__main__':
    app.run_server(debug = True, port = 8059)